In [10]:
# import_notebooks is a custom python script
# this line makes it possible to import functions from other IPython notebooks
from import_notebooks import *

In [11]:
# Here we import all the functions (*) from these notebooks
from Hainan_Data_Cleaning import *
from time_shift_dataframe import *
from Stratified_Split import *
from Cumulative import *
from Rollingsum import *
from LR_model import *
from Classification_Functions import *
from nn_model_keras import *

from Utilities import *

In [12]:
# load_hainan_data() is from Hainan_Data_Cleaning and it loads hainan data
df = load_hainan_data()
df_ts = timeshift_df(df, 15)

df_ts.columns.values
df_ts_relfeatures = df_ts.drop(['Year', 'Month', 'Day','BioCNG', 'LiquidFertilizer',
       'SolidFertilizer', 'WasteWater', 'SolidResidues','ProjectElectricity',
       'OfficeElectricity', 'Water', 'Diesel'], axis=1)

print(df_ts_relfeatures.shape[0])
df_ts_relfeatures = df_ts_relfeatures.apply(pd.to_numeric, errors='coerce')
df_ts_relfeatures = df_ts_relfeatures.dropna()
print(df_ts_relfeatures.shape[0])

# df_ts_relfeatures_roll = rollingsum(df_ts_relfeatures,10)



1413
1413


In [13]:
df = df_ts_relfeatures
df.head()


,RawBiogas,PigManure,Cassava,FishWastewater,KitchenFoodWaste,MunicipalFecalWaste,TeaWaste,ChickenWaste,Bagasse,AlcoholWaste,ChineseMedicineWaste,EnergyGrass,BananaFruitShafts,LemonWaste,Percolate,Other,NaOH,FeCl2,PAM,Defoamer
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,14.9,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# FEATURE ENGINEERING GOES HERE
df_ts_relfeatures.head()
df = df_ts_relfeatures.copy()
df.head()

columns = df.columns.tolist()
df["RawBiogas3"] = np.zeros(df.shape[0])
df["Bagasse3"] = np.zeros(df.shape[0])
df["PigManure3"] = np.zeros(df.shape[0])
new_columns = list(["RawBiogas3", "Bagasse3", "PigManure3"])
new_columns.extend(columns)
print(new_columns)
df = df[new_columns]
df.head()


for i in range(2,df["RawBiogas"].shape[0]):
    df["RawBiogas3"][i] = sum([df["RawBiogas"][i-j] for j in range(3)])
    df["Bagasse3"][i] = sum([df["Bagasse"][i-j] for j in range(3)])
    df["PigManure3"][i] = sum([df["PigManure"][i-j] for j in range(3)])

df.head(10)
df.drop(df.head(3).index,inplace=True)
df.head(10)
df.drop(["RawBiogas"], axis=1, inplace=True)
df.head(10)

df_final = df.copy()


['RawBiogas3', 'Bagasse3', 'PigManure3', 'RawBiogas', 'PigManure', 'Cassava', 'FishWastewater', 'KitchenFoodWaste', 'MunicipalFecalWaste', 'TeaWaste', 'ChickenWaste', 'Bagasse', 'AlcoholWaste', 'ChineseMedicineWaste', 'EnergyGrass', 'BananaFruitShafts', 'LemonWaste', 'Percolate', 'Other', 'NaOH', 'FeCl2', 'PAM', 'Defoamer']


In [15]:
x_train_r, y_train_r, x_test_r, y_test_r = test_train_for_regression(df_final, "RawBiogas3")
x_train_c, y_train_c, x_test_c, y_test_c = test_train_for_classification(df_final, "RawBiogas3", num_buckets = 7)
x_train_c_1, y_train_c_1, x_test_c_1, y_test_c_1 = test_train_for_classification(df_final, \
                                output_var="RawBiogas3", \
                                num_buckets = 7, one_hot=True)

print(x_train_r.shape,  y_train_r.shape, x_test_r.shape, y_test_r.shape)
# print(x_train_r.columns.values)

(1128, 21) (1128,) (282, 21) (282,)


In [16]:
# Linear Regression
lr = run_lm(x_train_r, y_train_r, x_test_r, y_test_r)

Starting linear regression model.
                            OLS Regression Results                            
Dep. Variable:             RawBiogas3   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.646
Method:                 Least Squares   F-statistic:                     99.09
Date:                Mon, 16 Apr 2018   Prob (F-statistic):          1.33e-236
Time:                        03:53:23   Log-Likelihood:                -12048.
No. Observations:                1128   AIC:                         2.414e+04
Df Residuals:                    1106   BIC:                         2.425e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

In [17]:
rr = run_ridge(x_train_r, y_train_r, x_test_r, y_test_r, alpha = 0.5)

# Classification
lm = LogisticModel(x_train_c, y_train_c, x_test_c, y_test_c)

dt = DecisionTree(x_train_c, y_train_c, x_test_c, y_test_c)

rf = RandomForest(x_train_c, y_train_c, x_test_c, y_test_c)

nb = NaiveBayes(x_train_c, y_train_c, x_test_c, y_test_c)


# Neural Net 
l_and_m = run_keras_nn(x_train_c_1, y_train_c_1, x_test_c_1, y_test_c_1, \
             epochs=10, hidden_layers=6, layer_neurons = 300, \
             activation='tanh', learning_rate=0.01)

The mean absolute error of ridge regression on training set is:  8491.02790171
The mean absolute error of ridge regression test set is:  8776.59020581
Starting keras model.
21
7
Epoch 1/10
1128/1128 [==============================] - 1s 978us/step - loss: 1.6137 - mean_absolute_error: 0.2100 - categorical_accuracy: 0.3404
Epoch 2/10
1128/1128 [==============================] - 0s 337us/step - loss: 1.4677 - mean_absolute_error: 0.1991 - categorical_accuracy: 0.3989
Epoch 3/10
1128/1128 [==============================] - 0s 314us/step - loss: 1.4516 - mean_absolute_error: 0.1971 - categorical_accuracy: 0.4034
Epoch 4/10
1128/1128 [==============================] - 0s 364us/step - loss: 1.3995 - mean_absolute_error: 0.1957 - categorical_accuracy: 0.4105
Epoch 5/10
1128/1128 [==============================] - 0s 319us/step - loss: 1.3990 - mean_absolute_error: 0.1945 - categorical_accuracy: 0.4220
Epoch 6/10
1128/1128 [==============================] - 0s 336us/step - loss: 1.4251 - mean_

In [18]:
print("Linear Regression - Mean Absolute Error ", "train:",lr[0], "test:",lr[1])
print("Ridge Regression - Mean Absolute Error ", "train:",lr[0], "test:",lr[1])
print("Logistic Model - Accuracy:", lm[1])
print("Decision Tree - Accuracy:", dt[1])
print("Random Forest - Accuracy:", rf[1])
print("Naive Bayes - Accuracy:", nb[1])
print("Neural Net - Mean Absolute Error:", l_and_m[1], " Categorial Accuracy:", l_and_m[2])

Linear Regression - Mean Absolute Error  train: 7874.99261695 test: 8166.41564612
Ridge Regression - Mean Absolute Error  train: 7874.99261695 test: 8166.41564612
Logistic Model - Accuracy: 0.439716312057
Decision Tree - Accuracy: 0.382978723404
Random Forest - Accuracy: 0.429078014184
Naive Bayes - Accuracy: 0.27304964539
Neural Net - Mean Absolute Error: 0.193797464383  Categorial Accuracy: 0.404255319994
